In [ ]:
import databricks.koalas as ks
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from databricks.koalas.config import set_option

In [ ]:
set_option("compute.ops_on_diff_frames", True)

In [ ]:
# Генерация случайной выборки точек на координатной плоскости 100x100: 70% - участвуют, из них 95% - маркируются по сумме координат, а оставльные 5% - рандомно
xs = list(np.arange(0, 100)) * 100
ys = [item for sublist in list(map(lambda x: [x] * 100, np.arange(0, 100))) for item in sublist]
points = list(zip(xs, ys))
np.random.shuffle(points)
involved_points = points[:int(len(points) * 0.7)]
involved_points_95 = involved_points[:int(len(involved_points) * 0.95)]
involved_points_5 = involved_points[-int(len(involved_points) * 0.05):]

In [ ]:
def label_points(fst, snd):
  if fst + snd < 30:
    return 'A'
  elif fst + snd < 65:
    return 'B'
  else:
    return 'C'
  
def random_label_points():
  rand = np.random.random()
  if rand < 1 / 3:
    return 'A'
  elif rand < 2 / 3:
    return 'B'
  else:
    return 'C'

# Маркировка данных
labeled_involved_points_95 = [(first, second, label_points(first, second)) for (first, second) in involved_points_95]
labeled_involved_points_5 = [(first, second, random_label_points()) for (first, second) in involved_points_5]
all_points = labeled_involved_points_95 + labeled_involved_points_5
knn_data = ks.DataFrame(all_points, columns=['x', 'y', 'attr'])
knn_data.cache()
knn_data.head()

In [ ]:
# Создание подмножеств классифицированных и некласифицированных данных
knn_classified_data = knn_data[:int(len(knn_data) * 0.9)]
knn_classified_data.cache()
knn_unclassified_data = knn_data[-int(len(knn_data) * 0.1):].drop('attr')
knn_unclassified_data.cache()
knn_unclassified_data.head()

In [ ]:
# Создание таблицы distanced - таблицы расстояний между классифицированными и неклассифицированными точками
cartesian_product = lambda left, right: left.assign(key=1).merge(right.assign(key=1), on='key').drop('key', 1)
distanced = cartesian_product(knn_unclassified_data, knn_classified_data).rename(columns={'x_x': 'x', 'y_x': 'y', 'x_y': 'x2', 'y_y': 'y2'})
distanced['distance'] = np.sqrt((distanced.x - distanced.x2) ** 2 + (distanced.y - distanced.y2) ** 2)
distanced.cache()
distanced.head()

In [ ]:
# Добавление к полученной таблице столбца distance_rn - порядкового номера числа расстояния (по возрастанию) для текущей пары x, y
distanced_row_numbered = distanced.copy()
distanced_row_numbered['distance_rn'] = distanced.groupby(by=['x', 'y'])['distance'].rank(method="first", ascending=True).astype(int)
distanced_row_numbered.cache()
distanced_row_numbered.head()

In [ ]:
# Выбрать все записи из полученной таблицы, distance_rn которых не превышает 5
top_n_selected = distanced_row_numbered[distanced_row_numbered["distance_rn"] <= 5]
top_n_selected.head()

In [ ]:
# Подсчет частоты встречаемости у соседей (поле attr_cnt)
top_n_attr_counted = top_n_selected[['x', 'y', 'attr']].copy()
top_n_attr_counted['attr_cnt'] = 0
top_n_attr_counted = top_n_attr_counted.groupby(by=top_n_attr_counted.columns[: -1].values, as_index=False).count()
top_n_attr_counted.cache()
top_n_attr_counted.head()

In [ ]:
# Добавление к полученной таблице столбца attr_rn - порядкового номера числа количества упоминаний attr у соседей для этой пары x, y 
top_n_attr_counted_and_numbered

In [ ]:
# Выбор самого часто встречаемого атрибута у соседей
classified_unclassified_data = top_n_attr_counted_and_numbered[top_n_attr_counted_and_numbered["attr_rn"] == 1][["x", "y", "attr"]]
classified_unclassified_data.cache()
classified_unclassified_data.head()

In [ ]:
# Подчсет точности классификации
diff_df = pd.merge(knn_data.to_pandas(), classified_unclassified_data.to_pandas(), how='outer', indicator='Exist')
diff_df = diff_df.loc[diff_df['Exist'] == 'both']
accuracy = len(diff_df) * 100 / len(knn_unclassified_data)
print(f"Точность классификации: {round(accuracy, 2)}%")